<h1 align ="center"> REST API Mutiple Image Samples</h1>
<hr>
   
# Chat Completions

In [1]:
import base64
%run shared_functions.ipynb

In [3]:

#Defect Detector
reference_image_file_path = "../../common/images/DefectDetectorReference.jfif" 
with open(reference_image_file_path, 'rb') as image_file:
    encoded_reference_image = base64.b64encode(image_file.read()).decode('utf-8')

test_image_file_path = "../../common/images/DefectDetectorTest.jfif"
with open(test_image_file_path, 'rb') as image_file:
    encoded_test_image = base64.b64encode(image_file.read()).decode('utf-8')

sys_message = """
You're a professional defect detector.
Your job is to compare the test image with reference image, please answer the question with "No defect detected" or "Defect detected", 
also explain your decision as detail as possible."
"""

messages = [
    {
      "role": "system",
      "content": [
          {
            "type": "text", 
            "text": sys_message
          }
      ]
    },
    {
      "role": "user",
      "content": [
          {
            "type": "text",
            "text": "Here is the reference image", # Pass the prompt
          },
          {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{encoded_reference_image}" #Pass the encoded reference image
            }
          },
          {
            "type": "text",
            "text": "Here is the test image", # Pass the prompt
          },
          {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{encoded_test_image}" #Pass the encoded test image
            }
          }
      ]
    }    
  ]

try:
    response_content = call_GPT4V_image(messages)
    print(response_content['choices'][0]['message']['content'])  # Print the content of the response
except Exception as e:
    raise SystemExit(f"Failed to call GPT-4V API. Error: {e}")

Defect detected.

The test image shows a can lid with visible deformations. There are areas where the material appears to be peeled or chipped off, especially around the center near the pull tab. This is a clear deviation from the reference image, which shows a flawless, smooth surface with no imperfections. The reference image displays a pristine can lid with an intact surface and an unblemished appearance, whereas the test image exhibits damage that would be considered a defect in a quality control assessment.
